In [1]:

####################################################################################################
####################################################################################################

import sys
import os

import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pprint

from IPython.display import clear_output
from IPython.display import display

sys.path.append(os.path.abspath("../"))
sys.path.append(os.path.abspath("../pax"))

from pax_utils import event_utils
from pax_utils import file_utils
from pax_utils import interaction_utils
from pax_utils import s1s2_utils
from pax_utils import waveform_utils

from pax import core

pp = pprint.PrettyPrinter(depth=6)



In [2]:

####################################################################################################
####################################################################################################

f = plt.figure(figsize=(20, 10))

def processPklEvents(zipfilename):

    ################################################################################################
    ################################################################################################
    
    lst_pkl_files  = file_utils.getListOfPklFilesFromZipFile(zipfilename, nEventsPerFile)

    
    ################################################################################################
    ################################################################################################
    
    sum_wf = None
    event  = None
    df_zip_merged = pd.DataFrame()
    
        
    ################################################################################################
    ################################################################################################
    
    for iPklFile in range(0, len(lst_pkl_files)):
    
        print("File: '" + str(zipfilename) + "'")
        print("   event_number: " + str(iPklFile))
        #clear_output(wait=True)
        
        
        ############################################################################################
        ############################################################################################
        
        pklfilename  = lst_pkl_files[iPklFile]
        event        = file_utils.getPaxEventFromPklFileInZipArchive(zipfilename, pklfilename)

      
        ############################################################################################
        ############################################################################################
        
        interactions  = event.interactions
        nInteractions = len(interactions)
        
        #if (nInteractions != 1): continue

            
            
        ############################################################################################
        ############################################################################################
        
        df_pkl_event  = event_utils.getEventDataFrameFromEvent(event)
        df_pkl_intr   = interaction_utils.getInteractionDataFrameFromEvent(event)
        df_pkl_s2s    = s1s2_utils.getS2integralsDataFrame(event, 127)
        
                
        ############################################################################################
        ############################################################################################
        
        left                        = event.main_s2.left
        right                       = event.main_s2.right
        jsonfilename                = os.path.dirname(zipfilename) + '/pax_info.json'
        cfg                         = event_utils.getConfig(jsonfilename)
        
        df_channels_waveforms_top   = waveform_utils.getChannelsWaveformsDataFrame(event, cfg, 'top', False)
        df_channels_waveforms_top   = waveform_utils.test(df_channels_waveforms_top)
        
        arr_summed_waveform_top_evt = waveform_utils.getSummedWaveformFromEvent(event, 'tpc_top')[left:right]
        arr_summed_waveform_top_df  = waveform_utils.getSummedWaveformFromDataFrame(df_channels_waveforms_top, event)[left:right]
        
        
        ############################################################################################
        # Check that the S2 summed waveform from the event and dataframe are equal
        ############################################################################################

        sum_evt       = np.sum(arr_summed_waveform_top_evt)
        sum_df        = np.sum(arr_summed_waveform_top_df)
        
        wf_arrs_equal = waveform_utils.compareArrays(arr_summed_waveform_top_evt, arr_summed_waveform_top_df)
        wf_sums_equal = waveform_utils.compareFloats(sum_evt, sum_df)
        
        assert(wf_arrs_equal and wf_sums_equal)
        
               
        ############################################################################################
        ############################################################################################
    
        ser = df_pkl_s2s.iloc[0]
        ser = ser[1:]
        ser = ser.as_matrix().astype(np.float32)
    
    
        ############################################################################################
        # Check that the per-channel S2 integrals from the event and dataframe are equal
        ############################################################################################
    
        arr_sum_evt = ser.astype(np.float32)
        arr_sum_df  = df_channels_waveforms_top[:]['sum'].as_matrix().astype(np.float32)
        
        sum_arr_evt = np.sum(arr_sum_evt)
        sum_arr_df  = np.sum(arr_sum_df)
    
        len_arr_evt = arr_sum_evt.size
        len_arr_df  = arr_sum_df.size
        
        arrs_equal = waveform_utils.compareArrays(arr_sum_evt, arr_sum_df)
        
        #assert(arrs_equal)
       
        
        ############################################################################################
        ############################################################################################
        
        print()
        print("Integral of Summed Waveform Event:       " + str(sum_evt))
        print("Integral of Summed Waveform DF:          " + str(sum_df))
        print()
        print("Length of S2 Integrals Array Event:      " + str(len_arr_evt))
        print("Length of S2 Integrals Array DF:         " + str(len_arr_df))
        print()
        print("Sum of S2 Integrals over channels Event: " + str(sum_arr_evt))
        print("Sum of S2 Integrals over channels DF:    " + str(sum_arr_df))
        print()
        
        display(df_channels_waveforms_top[0:5][:])
        
        
        
        arr = waveform_utils.getS2IntegralsFromDataFrame(df_channels_waveforms_top, event)
        
        print("check")
        print(np.sum(arr))
        print(arr)
        print()
        
        
        #############################################################################################
        #############################################################################################
        #
        #print()
        #
        #nTot = 0
        #
        #for i in range(0, arr_sum_evt.size):
        #    
        #    x1 = arr_sum_evt[i]
        #    x2 = arr_sum_df[i]
        #
        #    if (not waveform_utils.compareFloats(x1, x2, 1e-6, False)):
        #        
        #        #print()
        #        #print("channel: " + str(i))
        #        #waveform_utils.compareFloats(x1, x2, 1e-3)
        #
        #        nTot += 1
        #    
        #    continue
        #
        #print(nTot)
        #print()
        
        
        ############################################################################################
        ############################################################################################
    
        df_pkl_merged = df_pkl_event.merge(df_pkl_intr).merge(df_pkl_s2s)
        df_zip_merged = df_zip_merged.append(df_pkl_merged)
        
        continue
      
    
    ################################################################################################
    ################################################################################################
    
    df_zip_merged.reset_index(inplace=True, drop=True)
       
    return df_zip_merged

    

In [3]:
import glob

####################################################################################################
####################################################################################################

#ver = 'test'
ver = datetime.datetime.now().strftime("%y%m%d%H%M")
ver = 'test'

nEventsPerFile = 2 #1000
nFilesZip      = 1 #200
nEvents        = nEventsPerFile * nFilesZip
dir_input    = '../pax_run/pax_output/Mar25/'
dir_format   = dir_input + "instructions_" + ('[0-9]' * 6)
file_format  = 'XENON1T-0-000000000-000000999-000001000.zip'
file_pkl     = 'merged/merged_pax_' + str(nEvents % 1000) + 'k_' + ver + '.pkl'
lst_contents = glob.glob(dir_format)

if (nFilesZip == -1):
    
    nFilesZip    = len(lst_contents)

print()
print("PAX input files: " + str(nFilesZip))
print()


####################################################################################################
####################################################################################################

df = pd.DataFrame()

for iZip in range(0, nFilesZip):
    
    ################################################################################################
    ################################################################################################
    
    zipfilename = lst_contents[iZip] + '/' + file_format
    
    if (not os.path.exists(zipfilename)):
        
        print("Error! File: '" + str(zipfilename) + "' does not exist.")
    
        continue
        
    print("File: '" + str(zipfilename) + "'")
    
    
    ################################################################################################
    ################################################################################################
    
    df_zip_merged = processPklEvents(zipfilename)
    
    df = df.append(df_zip_merged)
    
    continue
    

print("Done")


####################################################################################################
# Write
####################################################################################################

print(file_pkl)

df.to_pickle(file_pkl)




PAX input files: 1

File: '../pax_run/pax_output/Mar25/instructions_000000/XENON1T-0-000000000-000000999-000001000.zip'
File: '../pax_run/pax_output/Mar25/instructions_000000/XENON1T-0-000000000-000000999-000001000.zip'
   event_number: 0
Top PMT's with zero gains: 15
[  1   2  12  26  27  34  62  65  73  79  86  88  91 102 118]
blah2: 260.56758287
blah3: 260.56758287

Integral of Summed Waveform Event:       260.568
Integral of Summed Waveform DF:          260.568

Length of S2 Integrals Array Event:      127
Length of S2 Integrals Array DF:         127

Sum of S2 Integrals over channels Event: 259.256
Sum of S2 Integrals over channels DF:    260.568



,channel,left,right,length,sum,raw_data
0,0,82932,83327,396,1.435604,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
0,1,0,0,0,0.000000,[]
0,2,0,0,0,0.000000,[]
3,3,82836,83005,170,2.190272,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,4,82886,83013,128,3.737455,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


row left: 82932
row left: 81914
row left: 82904
row left: 97648
row left: 97688
row left: 97702
row left: 100794
row left: 82848
row left: 82814
row left: 82902
row left: 82956
row left: 97642
row left: 97668
row left: 100812
row left: 82804
row left: 100810
row left: 83230
row left: 100794
row left: 82836
row left: 82886
check
300.033
[  1.43560374   0.           0.           2.19027233   3.73745537
   0.9848901    0.           0.           0.           6.69082451
   6.68689919   2.35583258   0.           0.81786031   0.           0.
   3.10084391   0.           0.           0.95770657   0.           0.           0.
   3.63397264   1.50121272   0.           0.           0.           0.           0.
   1.58797121   2.72301507   1.29693341   0.           0.           0.
   4.22049713  14.6941185   12.98080826  11.53015804   1.39322805
   1.23892021   3.53061342   0.58457142   2.22669411   2.66378117   0.           0.
   0.           0.           0.           1.40889943   0.           0.

,channel,left,right,length,sum,raw_data
0,0,105424,105791,368,1.715228,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
0,1,0,0,0,0.000000,[]
0,2,0,0,0,0.000000,[]
0,3,0,0,0,0.000000,[]
4,4,65458,148393,82936,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


row left: 65458
row left: 10394
row left: 13184
row left: 113038
row left: 105646
row left: 105538
row left: 105420
row left: 105566
row left: 105538
row left: 131042
row left: 105504
row left: 105650
row left: 115130
row left: 105642
row left: 5332
row left: 105568
row left: 105412
row left: 105428
row left: 105394
row left: 105542
row left: 105580
row left: 105508
row left: 105438
row left: 105570
row left: 105528
row left: 148292
row left: 105694
row left: 105430
row left: 12856
row left: 105394
row left: 105436
row left: 105540
row left: 105640
row left: 105472
row left: 105620
row left: 105408
row left: 105422
row left: 105380
row left: 105428
row left: 105534
row left: 105572
row left: 105404
row left: 105400
row left: 105504
check
779.656
[   1.71522796    0.            0.            0.            0.82767719
    1.03410113    0.            0.            0.            1.29310763
    1.38525522    0.            0.            0.            0.
    2.07203436    4.50166321    1.34905